In [2]:
import scipy.io as sio
mat_file = sio.loadmat("eventrain.mat")

In [1]:
import numpy as np
def mean_absolute_percentage_error(y_true, y_pred): 
  return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [18]:
def cercano(x,y,z,n):
    a=abs(x-n)
    b=abs(y-n)
    c=abs(z-n)
    if(a<=b and a<=c):
        return x
    elif(b<=a and b <=c):
        return y
    else:
        return z
    

In [6]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

importancion de librerias y se carga el archivo de entrenamiento

In [3]:
data = mat_file["trcoll"][0][0] # is "tecoll" for testing set
genFeat = data[0]   # Contextual features
ffcoefs = data[3]   # Fisherface space
faceGist = data[4]  # GIST features
edad=data[1]

Se carga la edad real en una variable y se mantienen 3 datos de entrenamiento por separado

In [5]:
np.random.seed(0)
mask_data=np.random.rand(edad.shape[0]) < 0.30

genF_train = genFeat[~mask_data]
genF_test = genFeat[mask_data]

ffcoefs_train = ffcoefs[~mask_data]
ffcoefs_test = ffcoefs[mask_data]

faceGist_train = faceGist[~mask_data]
faceGist_test = faceGist[mask_data]

edad_train = edad[~mask_data]
edad_test = edad[mask_data]

Se separan los datos para poder probar los modelos propuestos, se puede utilizar tambien eventest para probar en vez de separar como en este caso, pero de esta forma es mas facil probar

In [7]:
std = StandardScaler()

std.fit(genF_train)
genF_train = std.transform(genF_train) 
genF_test = std.transform(genF_test)

std.fit(ffcoefs_train)
ffcoefs_train = std.transform(ffcoefs_train) 
ffcoefs_test = std.transform(ffcoefs_test)

std.fit(faceGist_train)
faceGist_train = std.transform(faceGist_train) 
faceGist_test = std.transform(faceGist_test)



Se estandarizaron las matrices de los datos obtenidos a mano para poder usarlos en un ajuste lineal

In [8]:
from sklearn.linear_model import LinearRegression
from numpy import random as random
model = LinearRegression(fit_intercept=True)
model.fit(genF_train, edad_train)

edad_pred_train = model.predict(genF_train)
edad_pred_test = model.predict(genF_test)

residuales = list()
for i in range(len(edad_pred_train)):
    error = edad_train[i] - edad_pred_train[i]
    residuales.append(error)
#####################################################
model2 = LinearRegression(fit_intercept=True)
model2.fit(ffcoefs_train, edad_train)

edad_pred_train2 = model2.predict(ffcoefs_train)
edad_pred_test2 = model2.predict(ffcoefs_test)

residuales2 = list()
for i in range(len(edad_pred_train2)):
    error = edad_train[i] - edad_pred_train2[i]
    residuales2.append(error)

###################################################
    
model3 = LinearRegression(fit_intercept=True)
model3.fit(faceGist_train, edad_train)

edad_pred_train3 = model3.predict(faceGist_train)
edad_pred_test3 = model3.predict(faceGist_test)

residuales3 = list()
for i in range(len(edad_pred_train3)):
    error = edad_train[i] - edad_pred_train3[i]
    residuales3.append(error)

Se pone a entrenar con los valores, creando 3 modelos distintos los cuales se tendran que relacionar de alguna forma entre si para la solucion final. Tambien se puede buscar una forma de relacionarlos desde antes. Tambien se guarda el error residual para un posterior analisis.

In [21]:
l=[]
for i in range(len(residuales)):
    a=cercano(edad_pred_train[i],edad_pred_train2[i],edad_pred_train3[i],edad_train[i])
    l.append(a)

errorprom=error=mean_absolute_percentage_error(edad_train,l)
print(errorprom)

130.23917720808154


Guardando los valores mas cercanos al teorico se tiene un error muy grande, lo que quiere decir que es un mal metodo de prediccion y modelo, se debe buscar otra forma de relacionar los datos para poder tener un estimador real.

In [25]:
final=[]
for i in range(len(data)):
    if(i==1 or i==8):
        continue
    separacion=data[i][~mask_data]
    std.fit(separacion)
    separacion=std.transform(separacion)
    model = LinearRegression(fit_intercept=True)
    model.fit(separacion, edad_train)
    edad_pred=model.predict(separacion)
    final.append(edad_pred)

C:\Users\Gonzalo\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
C:\Users\Gonzalo\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype uint16 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


En este intento se crea una lista final, en la cual se iran almacenando los vectores obtenidos de las predicciones. Cada uno de las matrices pasa por el proceso de estandarizacion y se pasa por un nuevo modelo de entrenamiento. Se salta el 1 y el 8 por representar valores no utiles de entrenamiento

In [23]:
tf=[]
for i in range(len(edad_train)):
    suma=final[0][i]+final[1][i]+final[2][i]+final[3][i]+final[4][i]+final[5][i]+final[6][i]+final[7][i]+final[8][i]
    suma=suma/9
    tf.append(suma[0])
errorprom=error=mean_absolute_percentage_error(edad_train,tf)

De la forma descrita se tiene un error aun mayor, esto pasa debido a que en ciertas matrices se tienen valores muy grandes en las predicciones, por lo que habria que sacarlos del algoritmo final al ser datos que generan ruido. En este metodo se trato de usar el promedio de las predicciones de las distintas matrices.

La mejor forma parece ser relacionar los datos desde antes de la union o utilizar otro modelo de prediccion. La otra opcion ya fue mencionada, utilizando lo pedido en el inciso m de la pregunta 1 se puede hacer seleccion de atributos para poder utilizar solo las matrices que aporten al problema, dejando fuera las que generan datos de ruido.